# 3. Работа с Big Data

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Test") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

## 3.1. Spark SQL (каталог, метаданные)

Каталог (Catalog) — это абстракция в Spark SQL, которая предоставляет API для:
- Управления базами данных (databases)
- Работы с таблицами (tables)
- Доступа к представлениям (views)
- Хранения метаданных (схем, типов данных, партиций)

Встроенный каталог vs Hive Metastore
| Особенность | Встроенный каталог | Hive Metastore |
|- |- |- |
| Где хранятся метаданные? | В памяти (исчезают после завершения сессии) | Во внешней БД (MySQL, PostgreSQL) |
| Когда использовать? | Для временных таблиц | Для постоянных таблиц в продакшене |
| Доступ из других приложений | Нет| Да (через Hive, Impala и др.) |

Работа с каталогом через SparkSession

Показать все базы данных
`spark.catalog.listDatabases()`  
Показать все таблицы в текущей БД
`spark.catalog.listTables()`  
Показать все колонки таблицы
`spark.catalog.listColumns(df_Titanic)`

Создание и управление базами данных
Создать новую БД
`spark.sql("CREATE DATABASE IF NOT EXISTS my_db")`  
Переключиться на БД
`spark.catalog.setCurrentDatabase("my_db")`  
Удалить БД (осторожно!)
`spark.sql("DROP DATABASE IF EXISTS my_db CASCADE")`

Временные vs постоянные таблицы

| Тип | Временная таблица | Постоянная таблица |
|- |- |- |
| Видимость | Только в текущей сессии Spark | Доступна всем сессиям |
| Хранение метаданных | В памяти | В Hive Metastore |
| Хранение данных | Зависит от источника| Сохраняется на HDFS/S3 |

Временная таблица (исчезнет после завершения сессии)
`df.createOrReplaceTempView("temp_titanic")`  
Постоянная таблица (сохранится в каталоге Hive)
`df.write.saveAsTable("perm_titanic")`

***VIEW*** — это виртуальная таблица (запрос не выполняется, пока не вызвана).
***TABLE*** — материализованные данные.

```
# Временное представление
spark.sql("""
    CREATE OR REPLACE TEMP VIEW adult_passengers AS
    SELECT * FROM titanic WHERE Age >= 18
""")

# Постоянное представление (требует Hive)
spark.sql("""
    CREATE OR REPLACE VIEW perm_adult_passengers AS
    SELECT * FROM titanic WHERE Age >= 18
""")
```

In [5]:
# Чтение таблицы из Hive


## 3.2. Structured Streaming (основы)

## 3.2. Structured Streaming (основы)

## 3.3. Оптимизация запросов (Explain, Tungsten)

## 3.4. Работа с GraphFrames

## 3.5. Интеграция с Python (Pandas, NumPy)

In [3]:
spark.stop()